In [ ]:
# import argparse
# from dataclasses import dataclass, field
# from typing import Optional, List, Dict
# import sys
# import torch
# from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM

# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # 加载模型和分词器
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_path = "/kaggle/input/qwen2.5/transformers/0.5b/1"
# max_position_embeddings = 2048  # 模型支持的最大长度
# tokenizer = AutoTokenizer.from_pretrained(model_path,device_map="auto" )
# model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_path, 
#                                             torch_dtype="float32",
#                                             trust_remote_code=True,  # Qwen模型需要这个参数
#                                             device_map="auto" ).to(device)

# # 初始化对话历史记录
# conversation_history = []

# # 定义格式化对话的函数
# def format_conversation(history):
#     """
#     格式化对话历史，明确标注对话轮次和说话人。②
#     """
#     formatted_history = ""
#     for i, (speaker, text) in enumerate(history):
#         if i==len(history)-1:
#             break
#         formatted_history += f"[Round {i+1}] {speaker}: {text}\n"
#     formatted_history += f"above is the history of bot's talk with user. The response to {history[-1][1]} is "
#     return formatted_history

# # 定义生成回复的函数
# def generate_response(conversation_history, user_input):
#     """
#     根据用户输入和对话历史生成模型回复。
#     """
#     # 更新对话历史②
#     conversation_history.append(("User", user_input))
#     formatted_conversation = format_conversation(conversation_history)
#     # 对话长度裁剪逻辑③
#     while len(tokenizer(formatted_conversation)["input_ids"]) > max_position_embeddings:
#         # 如果超出最大长度，移除最旧的一轮对话
#         conversation_history.pop(0)
#         formatted_conversation = format_conversation(conversation_history)
    
#     # 编码输入并生成回复
    
#     inputs = tokenizer(formatted_conversation, return_tensors="pt", truncation=True, max_length=max_position_embeddings).to(device)
#     # generate 文档
#     outputs = model.generate(**inputs,pad_token_id=tokenizer.eos_token_id, #在生成时用eos填充序列
#                             max_new_tokens=50, #新生成文本长度
#                             # num_beams=5,
#                             # temperature=0.7,
#                             # top_k=50,
#                             # top_p=0.95,
#                             # repetition_penalty=1.2
#                             )
#     # print(outputs)
#     response = tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], skip_special_tokens=True) #在解码过程中跳过特殊符号如eos pad
    
#     # 添加对话历史②
#     conversation_history.append(("Bot", response.strip()))
#     return response.strip()

# # 定义主聊天逻辑
# def chat():
#     """
#     主聊天逻辑，支持 quit、newsession 等指令。
#     """
#     global conversation_history
#     print("Let's chat!")
#     while True:
#         # 获取用户输入
#         user_input = input("User: ").strip()
#         # 处理不同指令 ③
#         if user_input.lower() == "\quit":
#             print("Session ended. Bye!")
#             break
#         elif user_input.lower() == "\\newsession":
#             conversation_history = []
#             print("Conversation history cleaned.")
#         else:
#             response = generate_response(conversation_history, user_input)
#             print(f"Bot: {response}")



完了 https://qwen.readthedocs.io/zh-cn/latest/framework/Langchain.html

In [17]:
!pip install langchain==0.3.13
!pip install langchain-community==0.3.13
!pip install langchain-core==0.3.28
!pip install langchain-text-splitters==0.3.4
!pip install langchain-chroma==0.1.4
!pip install sentence-transformers==3.3.1

1308.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


1316.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


1323.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

Defaulting to user installation because normal site-packages is not writeable


1337.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


1345.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencompass 0.3.7 requires sentence-transformers==2.2.2, but you have sentence-transformers 3.3.1 which is incompatible.


In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
# from langchain.llms import Qwen
from langchain.prompts import ChatPromptTemplate


In [2]:
## 1 文档读取
# 读取csv，返回list
def load_csv(path):
    # 每条记录为一个元素
    # https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html
    loader = CSVLoader(
        file_path=path,
        encoding='utf-8',# 编码
        csv_args={
                'delimiter': ',',
                'quotechar': '"',
                'fieldnames': ['Index', 'Height', 'Weight'] # CSV 文件应该包含这三个字段
                }
    )
    data = loader.load()
    return data 
# 读取pdf，返回list
def load_pdf(path):
    # 是以每页为一个元素的
    loader = PyPDFLoader(path)
    pages = loader.load_and_split() #why
    return pages
def load_txt(path):
    # https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.text.TextLoader.html
    loader=TextLoader(file_path=path,encoding='utf-8',)
    data=loader.load()
    return data
    
## 2 文档切块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # 指定每个文本块的目标大小，这里设置为200个字符。
    chunk_overlap=50, # 指定文本块之间的重叠字符数，这里设置为50个字符。
    length_function=len, # 用于测量文本长度的函数，这里使用Python内置的`len`函数。
    is_separator_regex=False, # 指定`separators`中的分隔符是否应被视为正则表达式，这里设置为False，表示分隔符是字面字符。
    separators=["\n\n",  "\n",   " ",    ".",    ",",     "，",  "。", ] # 定义用于分割文本的分隔符列表。
)
pages = load_txt("/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/dataset/hlm.txt")
# texts = text_splitter.split_documents([pages[0].page_content])
texts = text_splitter.split_documents(pages)
# type(texts)=list
# texts[int]={page_content="...",metadata={'source': '/kaggle/input/the-dream-of-red-mansion/hlm.txt'}}


In [8]:
## 3 向量化
# reference: https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.huggingface.HuggingFaceBgeEmbeddings.html
# model_name = "BAAI/bge-large-en-v1.5"
import torch
print(f"CUDA available: {torch.cuda.is_available()}")

model_name = "BAAI/bge-large-zh"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # 当向量都被规范化（归一化）后，它们的范数都是1。

# model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
# encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# embedding = hf.embed_query("贾宝玉是谁")
# print(embedding)



CUDA available: True


In [9]:
## 4 创建向量数据库
# 快速创建数据库
# reference: https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html#langchain_community.vectorstores.chroma.Chroma.from_documents
vectorstore = Chroma.from_documents(
   documents = texts, 
   embedding = hf, #第三步得到的用于计算embedding的模型
   ids = None,
   collection_metadata = {"hnsw:space": "cosine"}, # 算余弦相似度
   )

# 相似度方法通过查询文本检索数据 示例
query = "贾宝玉是谁"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

.因长得齐整，老太太很疼.自幼淘气异常，天天逃学，老爷太太也不便十分管教。”贾母笑道：“也不成了我们家的了！你这哥儿叫什么名字？"四人道：“因老太太当作宝贝一样，他又生的白，老太太便叫作宝玉。”贾母便向李纨等道：“偏也叫作个宝玉。”李纨忙欠身笑道：“从古至今，同时隔代重名的很多。”四人也笑道：“起了这小名儿之后，我们上下都疑惑，不知那位亲友家也倒似曾有一个的


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from abc import ABC
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun

model_name = "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/peft_output/checkpoint-32350"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

class Qwen(LLM, ABC):
     max_token: int = 10000
     temperature: float = 0.01
     history_len: int = 3

     def __init__(self):
         super().__init__()

     @property
     def _llm_type(self) -> str:
         return "Qwen"

     @property
     def _history_len(self) -> int:
         return self.history_len

     def set_history_len(self, history_len: int = 10) -> None:
         self.history_len = history_len

     def _call(
         self,
         prompt: str,
         stop: Optional[List[str]] = None,
         run_manager: Optional[CallbackManagerForLLMRun] = None,
     ) -> str:
         messages = [
             {"role": "system", "content": "你是红楼梦的智能助手，你回答的问题要符合红楼梦的背景。"},
             {"role": "user", "content": prompt}
         ]
         text = tokenizer.apply_chat_template(
             messages,
             tokenize=False,
             add_generation_prompt=True
         )
         model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
         generated_ids = model.generate(
             **model_inputs,
             max_new_tokens=512
         )
         generated_ids = [
             output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
         ]

         response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
         return response

     @property
     def _identifying_params(self) -> Mapping[str, Any]:
         """Get the identifying parameters."""
         return {"max_token": self.max_token,
                 "temperature": self.temperature,
                 "history_len": self.history_len}

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.25s/it]


In [19]:
## 5
# 创建一个检索器 https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html#langchain_community.vectorstores.chroma.Chroma.as_retriever
from langchain_core.output_parsers.string import StrOutputParser
from typing import Sequence
from langchain.schema import Document
from langchain.schema.document import Document

retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # k 表示要检索的结果数量
# query = "萧炎是谁?"
# https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html#langchain_core.vectorstores.base.VectorStoreRetriever
query = "你好"
docs = retriever.invoke(query)

# 实例化自定义模型
llm = Qwen()
PROMPT_TEMPLATE = "请根据提示回答问题"
template = """基于下列红楼梦的背景，回答问题。
红楼梦的背景：{context}
问题：{question}
"""

def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(doc_string)
    return "\n".join(formatted_docs)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html

output_parser = StrOutputParser()
# 构建 chain
chain = prompt | llm | output_parser
res = chain.invoke(
                    {
                    "context": format_docs(docs),
                    "question": query
                    }
                    )

print(res)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Assistant:很抱歉，我无法回答这个问题。因为提示中没有提供足够的信息，我无法理解您的问题。如果您能提供更多的信息，我将尽力为您提供帮助。


In [17]:
print(prompt)

input_variables=[] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='请根据提示回答问题'), additional_kwargs={})]
